In [28]:
from docx import Document
import os
import glob
import pandas as pd
import spacy
from spacy.pipeline import EntityRuler

In [29]:
# Função para extrair texto de um arquivo .docx
def extract_text(docx_file):
    document = Document(docx_file)
    full_text = []
    for para in document.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

In [31]:
	
# Listas para armazenar os textos
textos_pre = []
textos_em_correcao = []
textos_pos = []

# Diretório onde os arquivos estão armazenados
diretorio = '../peticoes'

# Navegar até o diretório
os.chdir(diretorio)

In [32]:
# Verificar se o número de arquivos em cada lista é igual
arquivos_pre = sorted(glob.glob('preCorrecao*.docx'))
arquivos_em_correcao = sorted(glob.glob('duranteCorrecao*.docx'))
arquivos_pos = sorted(glob.glob('posCorrecao*.docx'))
assert len(arquivos_pre) == len(arquivos_em_correcao) == len(arquivos_pos), "Número de arquivos inconsistente."

In [33]:
# verificando arquivos encontrados
print(f'Arquivos pré-correção: {arquivos_pre}')
print(f'Arquivos em correção: {arquivos_em_correcao}')
print(f'Arquivos pós-correção: {arquivos_pos}')


Arquivos pré-correção: ['preCorrecao1.docx', 'preCorrecao2.docx', 'preCorrecao3.docx', 'preCorrecao4.docx']
Arquivos em correção: ['duranteCorrecao1.docx', 'duranteCorrecao2.docx', 'duranteCorrecao3.docx', 'duranteCorrecao4.docx']
Arquivos pós-correção: ['posCorrecao1.docx', 'posCorrecao2.docx', 'posCorrecao3.docx', 'posCorrecao4.docx']


In [34]:
# Iterar pelos arquivos e extrair os textos
for arquivo_pre, arquivo_em_correcao, arquivo_pos in zip(arquivos_pre, arquivos_em_correcao, arquivos_pos):
    texto_pre = extract_text(arquivo_pre)
    texto_em_correcao = extract_text(arquivo_em_correcao)
    texto_pos = extract_text(arquivo_pos)
    
    textos_pre.append(texto_pre)
    textos_em_correcao.append(texto_em_correcao)
    textos_pos.append(texto_pos)

In [35]:
	
# Criar um DataFrame para organizar os dados
df_peticoes = pd.DataFrame({
    'texto_pre': textos_pre,
    'texto_em_correcao': textos_em_correcao,
    'texto_pos': textos_pos
})
# Exibir o DataFrame
print(df_peticoes.head())

                                           texto_pre  \
0  São Paulo, 02 de setembro de 2024.\n\n\nAo\nIl...   
1  EXCELENTÍSSIMO JUIZ DA SEGUNDA ZONA ELEITORAL ...   
2  EXCELENTÍSSIMO JUIZ DE DIREITO DA DÉCIMA SEGUN...   
3  AInt no AREsp nº 2.602.095/SP\n\nAgravantes: E...   

                                   texto_em_correcao  \
0  São Paulo, 02 de setembro de 2024.\n\n\nAo\nIl...   
1  EXCELENTÍSSIMO JUIZ DA SEGUNDA ZONA ELEITORAL ...   
2  EXCELENTÍSSIMO JUIZ DE DIREITO DA DÉCIMA SEGUN...   
3  AInt no AREsp nº 2.602.095/SP\n\nAgravantes: A...   

                                           texto_pos  
0  São Paulo, 02 de setembro de 2024.\n\n\nAo\nIl...  
1  EXCELENTÍSSIMO JUIZ DA SEGUNDA ZONA ELEITORAL ...  
2  EXCELENTÍSSIMO JUIZ DE DIREITO DA DÉCIMA SEGUN...  
3  AInt no AREsp nº 2.602.095/SP\n\nAgravantes: E...  


In [170]:
patterns = [
    {
        "label": "NUM_OAB",
        "pattern": [
            {"TEXT": {"REGEX": r"OAB[/-][A-Z]{2}"}},  
            {"LOWER": "nº"},
            {"TEXT": {"REGEX": r"\d{3}\.\d{3}"}}
        ]
    },
    {
    "label": "CPF",
    "pattern": [{"TEXT": {"REGEX": r"\d{3}\.\d{3}\.\d{3}-\d{2}"}}]
    },
    
    {
    "label": "CNPJ",
    "pattern": [{"TEXT": {"REGEX": r"\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}"}}]
    },
    {
    "label": "RG",
    "pattern": [{"TEXT": {"REGEX": r"\d{2}\.\d{3}\.\d{3}-\d{1}"}}]
    },
    {
    "label": "RG",
    "pattern": [{"TEXT": {"REGEX": r"\d{2}\.\d{3}\.\d{3}-\d{1}"}}]
}, 
    {
    "label": "TELEFONE",
    "pattern": [{"TEXT": {"REGEX": r"\(?\d{2}\)?\s?\d{4,5}-?\d{4}"}}]
},
    {
    "label": "ENDERECO",
    "pattern": [
        {"LOWER": {"IN": ["rua", "avenida", "travessa", "alameda", "praça"]}},
        {"IS_TITLE": True, "OP": "+"},
        {"LOWER": "nº"},
        {"LIKE_NUM": True},
        {"IS_PUNCT": True},
        {"IS_TITLE": True, "OP": "+"},
        {"LOWER": "cep"},
        {"TEXT": {"REGEX": r"\d{5}-\d{3}"}}
    ]
},
       # Data de Evento
    {
        "label": "DATA_EVENTO",
        "pattern": [{"TEXT": {"REGEX": r"\d{1,2} de [a-zç]+ de \d{4}"}}]
    },
    {
        "label": "DATA_EVENTO",
        "pattern": [{"TEXT": {"REGEX": r"\d{2}/\d{2}/\d{4}"}}]
    },
     # Padrões para EMAIL
    {
        "label": "EMAIL",
        "pattern": [
            {"LOWER": "e-mail"},
            {"ORTH": ":"},
            {"TEXT": {"REGEX": r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"}}
        ]
    },
    {
        "label": "EMAIL",
        "pattern": [{"TEXT": {"REGEX": r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"}}]
    }
    
    

]

In [171]:
nlp = spacy.load('pt_core_news_sm')

# Adicionar o EntityRuler ao pipeline usando o nome da fábrica
ruler = nlp.add_pipe("entity_ruler", before='ner')

# Adicionar os padrões ao EntityRuler
ruler.add_patterns(patterns)

In [172]:
# Função para anonimizar o texto
def anonimizar_texto(texto):
    doc = nlp(texto)
    texto_anonimizado = texto
    for ent in doc.ents:
        if ent.label_ in ['NUM_OAB', 'CPF', 'CNPJ', 'RG', 'TELEFONE', 'ENDERECO', 'DATA_EVENTO', 'PER', 'ORG', 'EMAIL', 'NUM_PROCESSO']:
            placeholder = f'[{ent.label_}]'
            texto_anonimizado = texto_anonimizado.replace(ent.text, placeholder)
    return texto_anonimizado


In [173]:
df_peticoes['texto_pre_anonimizado'] = df_peticoes['texto_pre'].apply(anonimizar_texto)
df_peticoes['texto_em_correcao_anonimizado'] = df_peticoes['texto_em_correcao'].apply(anonimizar_texto)
df_peticoes['texto_pos_anonimizado'] = df_peticoes['texto_pos'].apply(anonimizar_texto)

# variaveis para nao confundir

pre_anonimizado = df_peticoes['texto_pre_anonimizado']
durante_anonimizado = df_peticoes['texto_em_correcao_anonimizado']
pos_anonimizado = df_peticoes['texto_pos_anonimizado']


In [175]:
#printar anonimização dos textos
print(pre_anonimizado[0])

São Paulo, 02 de setembro de 2024.


Ao
Ilmo. [PER] (“[PER]”)

Ref: [PER] ([DATA_EVENTO]) e Proposta ([DATA_EVENTO])

A/C.: Dr. Luis Bitetti – OAB/SP nº 84.009

[EMAIL] 

Prezado Senhor,

			Na qualidade de advogados da BRL TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A., cessionária dos honorários contratuais então titulados pelo escritório [ORG] [PER], Dra. HALBA MERY PEREBONI ROCCO e Dr. HENRIQUE LINDENBOJM, em resposta ao parecer e à proposta em referência, e considerando que:

			(a) a ora Notificante adquiriu de boa-fé os créditos de honorários devidos por décadas de patrocínio da ação de [PER] nº 0527173-63.1994.4.03.6100, cujos requisitórios foram expedidos de forma apartada da verba principal;


			(b) Tal verba, de natureza alimentar (Súmula Vinculante 47/STF), é autônoma e não possui relação com a indenização devida pela União em decorrência da desapropriação de áreas situadas no município de Ubatuba/SP (Ubatumirim) na década de setenta para a expansão de rodovia f